In [1]:
import time
import pandas as pd
import sys
import os
import csv
import imp
import operator

import numpy as np
import nltk

from sklearn import linear_model, metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# train model using training set
# test model using test set output results to file
input_cols = ['X','y']

def load_file(file_name):
    return pd.read_csv(file_name, sep=',')

test_set = load_file('TestDatav1.csv')
training_set = load_file('TraningDatav1.csv')

In [3]:
training_set.head(5)

,Date,Opinion,Pro,TweetText
0,10/19/18 9:55,Positive,NaN,A new home brings with it the power to dream b...
1,10/19/18 9:52,Positive,NaN,Pradhan Mantri Awas Yojana beneficiaries from ...
2,10/18/18 11:29,Neutral,NaN,Saddened by the passing away of Shri ND Tiwari...
3,10/17/18 14:30,Negative,Pro Gov,The Madhya Pradesh Congress is \xe2\x80\x98ama...
4,10/17/18 3:05,Positive,NaN,"Today we worship Maa Mahagauri, the 8th Avatar..."


In [4]:
test_set.head(5)

,Date,Opinion,Pro,TweetText
0,3/16/19 3:30,Positive,Pro Gov,Your Chowkidar is standing firm &amp; serving ...
1,3/15/19 13:02,Positive,Pro Gov,"This is a New India, where a person\xe2\x80\x9..."
2,3/15/19 12:57,Positive,Pro Gov,Public transport is convenient and comfortable...
3,3/15/19 12:51,Positive,NaN,Happy to see this! https://t.co/26gvVum5T9'
4,3/15/19 12:50,Positive,Pro Gov,Honoured that we have got the opportunity to w...


In [5]:
training_set.dropna(axis=0, how='any')

,Date,Opinion,Pro,TweetText
3,10/17/18 14:30,Negative,Pro Gov,The Madhya Pradesh Congress is \xe2\x80\x98ama...
15,10/6/18 12:43,Negative,Pro Gov,When I had to decide where to mark the Parakra...
26,9/25/18 13:41,Negative,Pro Gov,"It has been 17 years and counting, the Congres..."
27,9/25/18 13:36,Negative,Pro Gov,We have also lost elections but we have never ...
31,9/19/18 14:56,Positive,Pro Gov,Had a delightful interaction with Anganwadi wo...
43,9/14/18 14:32,Positive,Pro Gov,Wonderfully articulated and great that you are...
62,8/23/18 12:07,Positive,Pro Gov,"Today, over a lakh women from Gujarat got Raks..."
82,7/21/18 4:04,Negative,Pro Gov,Congress leaders feel I cant see them in the e...
83,7/21/18 4:03,Negative,Pro Gov,"Before net banking was popular, there was 'pho..."
84,7/21/18 4:02,Negative,Pro Gov,I appeal to the Congress to stop calling the S...


In [6]:
# helper functions:
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    # Use lemmatize
    #stems = lemmatize_tokens(tokens, lemmatizer)
    # Use stemming
    stems = stem_tokens(tokens, stemmer)
    return stems

stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens, lemmatizer):
    lemmatized = []
    for item in tokens:
        lemmatized.append( lemmatizer.lemmatize( item ) )
    return lemmatized




In [8]:

# train model:
def train(train_text, train_labels):
    global bow_transformer, tfidf_transformer 
    #model = linear_model.LogisticRegression(C=10, class_weight='balanced', fit_intercept=0, multi_class='ovr', solver='liblinear')
   
    model = SVC(C=0.9, kernel='linear', probability=True)
    #model = LinearSVC(C=1, class_weight= 'balanced', dual=True, fit_intercept=0, loss= 'hinge', multi_class='ovr',
#                     penalty='l2')


    X_train = train_text
    y_train = train_labels
   
    bow_transformer = CountVectorizer(ngram_range=(1, 3), analyzer='word', stop_words='english', tokenizer= tokenize).fit(X_train)

    X_train_dtm = bow_transformer.transform(X_train)     
    tfidf_transformer = TfidfTransformer().fit(X_train_dtm)
    X_train_dtm = tfidf_transformer.transform(X_train_dtm)
    
    return model.fit(X_train_dtm, y_train)

In [9]:
trained_model = train(training_set['TweetText'], training_set['Opinion'])

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

In [10]:
# prep test data manually, and return metrics
vect = CountVectorizer(ngram_range=(1, 3), stop_words='english', analyzer='word', tokenizer= tokenize)

analyze = vect.build_analyzer()
bow4 = bow_transformer.transform(test_set['TweetText'])
tfidf4 = tfidf_transformer.transform(bow4)

y_predit_class = trained_model.predict(tfidf4)
prec_rec_f1 = metrics.precision_recall_fscore_support(test_set['Opinion'], y_predit_class, average='weighted')
print(prec_rec_f1)
print('precision '+ str(prec_rec_f1[0]))
print('recall ' + str(prec_rec_f1[1]))
print('f1 ' + str(prec_rec_f1[2]))

(0.7769829592255855, 0.7303664921465969, 0.7137774751268284, None)
precision 0.7769829592255855
recall 0.7303664921465969
f1 0.7137774751268284


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
test_set['predicted'] = test_set['TweetText'].apply(lambda text: test(trained_model, text) )

NameError: name 'test' is not defined

In [36]:
test_set['accurate'] = np.where(test_set['Opinion'] == test_set['predicted'],1,0)

In [37]:
# export to .csv
file_name = "error_analysis_reg"  + time.strftime("%Y%m%d-%H%M%S") + '.csv'
with open(file_name, "w+") as to_file:
    test_set.to_csv(file_name, index=False, header=False, columns=['TweetText','Opinion','predicted','accurate'])

In [38]:
test_set.head(50)

,Date,Opinion,Pro,TweetText,predicted,accurate
0,3/16/19 3:30,Positive,Pro Gov,Your Chowkidar is standing firm &amp; serving ...,Negative,0
1,3/15/19 13:02,Positive,Pro Gov,"This is a New India, where a person\xe2\x80\x9...",Negative,0
2,3/15/19 12:57,Positive,Pro Gov,Public transport is convenient and comfortable...,Positive,1
3,3/15/19 12:51,Positive,NaN,Happy to see this! https://t.co/26gvVum5T9',Positive,1
4,3/15/19 12:50,Positive,Pro Gov,Honoured that we have got the opportunity to w...,Positive,1
5,3/15/19 12:49,Positive,Pro Gov,Thanks for the faith in our work. https://t.co...,Positive,1
6,3/14/19 16:09,Neutral,NaN,Deeply anguished by the loss of lives due to t...,Neutral,1
7,3/14/19 13:23,Positive,Pro Gov,Joined the ceremony to mark the presentation o...,Positive,1
8,3/14/19 13:18,Positive,NaN,"India First, India always! https://t.co/XKXkcb...",Negative,0
9,3/14/19 13:18,Positive,NaN,"Great gesture! \n\nAlso, Kudos to the great Da...",Positive,1
